In [0]:
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AKIAQYEI44NWS57Y5***")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "C4kc0VeRHGMg8BUEGyaZS4MoADm+oD5lMDaHo***")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.eu-west-1.amazonaws.com")

s3_path = "s3a://kinesis-s3-databrick/2025/04/10/*"

from pyspark.sql.types import StructType

schema = StructType() \
    .add("index", "integer") \
    .add("unique_id", "string") \
    .add("title", "string") \
    .add("description", "string") \
    .add("poster_name", "string") \
    .add("follower_count", "string") \
    .add("tag_list", "string") \
    .add("is_image_or_video", "string") \
    .add("image_src", "string") \
    .add("downloaded", "integer") \
    .add("save_location", "string") \
    .add("category", "string")

print("📥 Reading JSON data from S3...")
df_raw = spark.read.json(s3_path, schema=schema)
display(df_raw)

📥 Reading JSON data from S3...


index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
2386,38c3e021-527e-4da6-8d43-c0aa07500d12,Christmas Five Senses Book,"The sights, sounds and smells of Christmas stay with many of us for a lifetime. Perhaps cinnamon in your nasal cavity automatically takes you back to Grandma’s kitchen. Or at th…",Life Over C's,79k,"Christmas Books,Christmas Themes,Kids Christmas,Xmas,Christmas Projects,Preschool Christmas Crafts,Christmas Activities,Kindergarten Christmas,Winter Activities",image,https://i.pinimg.com/originals/3f/f7/54/3ff754d879c31507d1dc78104af06a63.jpg,1,Local save in /data/christmas,christmas
2386,38c3e021-527e-4da6-8d43-c0aa07500d12,Christmas Five Senses Book,"The sights, sounds and smells of Christmas stay with many of us for a lifetime. Perhaps cinnamon in your nasal cavity automatically takes you back to Grandma’s kitchen. Or at th…",Life Over C's,79k,"Christmas Books,Christmas Themes,Kids Christmas,Xmas,Christmas Projects,Preschool Christmas Crafts,Christmas Activities,Kindergarten Christmas,Winter Activities",image,https://i.pinimg.com/originals/3f/f7/54/3ff754d879c31507d1dc78104af06a63.jpg,1,Local save in /data/christmas,christmas
null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null


In [0]:
from pyspark.sql.functions import regexp_replace, col, trim, lower

print("Cleaning Data...")
df_cleaned = df_raw \
    .withColumn("follower_count", regexp_replace("follower_count", "k", "000")) \
    .withColumn("follower_count", regexp_replace("follower_count", "M", "000000")) \
    .withColumn("follower_count", col("follower_count").cast("int")) \
    .withColumn("category", trim(lower(col("category")))) \
    .dropDuplicates(["unique_id"]) \
    .na.drop()

display(df_cleaned)


🧹 Cleaning Data...


index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
2386,38c3e021-527e-4da6-8d43-c0aa07500d12,Christmas Five Senses Book,"The sights, sounds and smells of Christmas stay with many of us for a lifetime. Perhaps cinnamon in your nasal cavity automatically takes you back to Grandma’s kitchen. Or at th…",Life Over C's,79000,"Christmas Books,Christmas Themes,Kids Christmas,Xmas,Christmas Projects,Preschool Christmas Crafts,Christmas Activities,Kindergarten Christmas,Winter Activities",image,https://i.pinimg.com/originals/3f/f7/54/3ff754d879c31507d1dc78104af06a63.jpg,1,Local save in /data/christmas,christmas


In [0]:
from pyspark.sql.functions import regexp_replace, col, trim, lower

df_pin_cleaned = spark.sql("SELECT * FROM pin_raw") \
    .withColumn("follower_count", regexp_replace("follower_count", "k", "000")) \
    .withColumn("follower_count", regexp_replace("follower_count", "M", "000000")) \
    .withColumn("follower_count", col("follower_count").cast("int")) \
    .withColumn("category", trim(lower(col("category")))) \
    .dropDuplicates(["unique_id"]) \
    .na.drop()
display(df_cleaned)